In [0]:
import sys, os, shutil
from glob import glob
import numpy as np

## Change Runtime on Colab

## Install Packages

Following packages need to installed everytime. Also first change directory to the location where this notebook is located. 

In [7]:
# os.chdir('drive/workspace/Dogs-Cats-Redux/')
!pip install tqdm
!pip install bcolz
!pip install theano

    100% |████████████████████████████████| 61kB 3.0MB/s 
    100% |████████████████████████████████| 1.3MB 969kB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/e9/84/eb/f8f3caa627bb01ebc96034c3411f59870951246e5873b3f4c7
Successfully built bcolz
    100% |████████████████████████████████| 2.8MB 489kB/s 
  Running setup.py bdist_wheel for theano ... - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/46/a2/7d/b4cac381d5151daa9f9e0b3e4e4b65edaea6355ae296c97cf2
Successfully built theano


In [0]:
# first install libgpu from connect.ipynb
!pip install keras==1.2

In [9]:
%matplotlib inline
import sys, os, shutil
from glob import glob
import numpy as np
from utils import *
from vgg16 import Vgg16
from PIL import Image
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from IPython.display import FileLink

Using TensorFlow backend.


In [0]:
# Check where your data is before you and your pwd

DATA_DIR = '/content/data/redux/'
val_path = DATA_DIR + 'valid/'
train_path = DATA_DIR + 'train/'
test_path = DATA_DIR + 'test/'
results_path = DATA_DIR+ 'results/'

In [0]:
g = glob(train_path + '*.jpg')
g = np.random.permutation(g)
for i in range(2000):
    shutil.move(g[i], val_path)

In [0]:
g_cat = glob(train_path+'cat*.jpg')
g_dog = glob(train_path+'dog*.jpg')
for img in g_cat:
    shutil.move(img, train_path+'cats/')
for img in g_dog:
    shutil.move(img, train_path+'dogs/')

g_cat = glob(val_path+'cat*.jpg')
g_dog = glob(val_path+'dog*.jpg')
for img in g_cat:
    shutil.move(img, val_path+'cats/')
for img in g_dog:
    shutil.move(img, val_path+'dogs')



Now that we have organised our data lets finetune VGG16 model. 

## Finetune

In [0]:
batch_size=64
no_epochs=3

In [16]:
vgg = Vgg16()
train_path = '/content/data/redux/train'
train_batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(val_path, batch_size=batch_size)
vgg.finetune(train_batches)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
latest_weights_filename = None
for epoch in range(no_epochs):
    print ("\nRunning Epoch :  {0}".format(epoch))
    vgg.fit(train_batches, val_batches, nb_epoch=1)
    latest_weights_filename = "ft{0}.h5".format(epoch)
    vgg.model.save_weights(results_path+latest_weights_filename)
print ("Completed {0} fit operation\n".format(no_epochs))

In [0]:
for epoch in range(no_epochs):
    print ("Epoch {0}".format(epoch))
    vgg.model.load_weights(filepath=results_path+'ft{0}.h5'.format(epoch))
    test_batches, test_preds = vgg.test(test_path, batch_size=batch_size)
    isdog = test_preds[:,1]
    isdog = isdog.clip(min=0.05, max=0.95)
    test_filenames = test_batches.filenames
    test_file_ids = [int(f[8:-4]) for f in test_filenames]
    subm = np.stack([test_file_ids, isdog], axis=1)
    subm_filename = "submission{0}.csv".format(epoch)
    np.savetxt(subm_filename, subm, fmt='%d,%0.5f', header='id,label', comments='')

Epoch 0
Found 12500 images belonging to 1 classes.
Epoch 1
Found 12500 images belonging to 1 classes.
Epoch 2
Found 12500 images belonging to 1 classes.


In [0]:
FileLink('submission3.csv')

/home/akshay/workspace/redux/submission3.csv